In [1]:
from google.colab import drive
drive.mount('mnt')
!cp "mnt/My Drive/Colab Notebooks/installer.ipynb" . 
%run 'installer.ipynb'

Mounted at mnt
     |████████████████████████████████| 40kB 5.0MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48382 sha256=8cfc95d917d63daf9a899d9a7877294695d5c2e5fe5fe58322d22f1fa1751f4d
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
Cloning into 'lib/pybullet-gym'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 746 (delta 1), reused 1 (delta 0), pack-reused 740
Receiving objects: 100% (746/746), 19.30 MiB | 31.98 MiB/s, done.
Resolving deltas: 100% (410/410), done.
Obtaining file:///content/lib/pybullet-gym
     |████████████████████████████████| 76.6MB 43kB/s 
  Running setup.py develop for pybulletgym


In [ ]:
restart_runtime()

In [2]:
!cp "mnt/My Drive/Colab Notebooks/base_setup.ipynb" . 
%run 'base_setup.ipynb'

In [3]:
class Critic_TD3_DDPG(Network):
    def __init__(self, state_dims, action_dims, fc1_dim, fc2_dim, name='critic', chpt_dir='ddpg'):
        super(Network, self).__init__()
#         Network settings
        self.layers = nn.ModuleList().to(device)
#         Checkpoint system
        self.checkpoint_file = os.path.join(chpt_dir, name+'.h5')
        if not os.path.exists(chpt_dir):
            os.makedirs(chpt_dir)
#         Critic Network 1
        self.layers.append(
            nn.Linear(state_dims, fc1_dim)
        )
        self.layers.append(
            nn.Linear(fc1_dim + action_dims, fc2_dim)
        )
        self.layers.append(
            nn.Linear(fc2_dim, 1)
        )
#         Critic Network 2
        self.layers.append(
            nn.Linear(state_dims, fc1_dim)
        )
        self.layers.append(
            nn.Linear(fc1_dim + action_dims, fc2_dim)
        )
        self.layers.append(
            nn.Linear(fc2_dim, 1)
        )
        self.to(device)
    
    def forward(self, state, action):
        q1 = F.relu(self.layers[0](state))
        q1 = F.relu(self.layers[1](T.cat([q1, action], 1)))
        q1 = self.layers[2](q1)

        q2 = F.relu(self.layers[3](state))
        q2 = F.relu(self.layers[4](T.cat([q2, action], 1)))
        q2 = self.layers[5](q2)

        return q1, q2

    def Q1(self, state, action):
        q1 = F.relu(self.layers[0](state))
        q1 = F.relu(self.layers[1](T.cat([q1, action], 1)))
        q1 = self.layers[2](q1)

        return q1

In [4]:
class Critic_TD3(Network):
    def __init__(self, state_dims, action_dims, fc1_dim, fc2_dim, name='critic', chpt_dir='ddpg'):
        super(Network, self).__init__()
#         Network settings
        self.layers = nn.ModuleList().to(device)
#         Checkpoint system
        self.checkpoint_file = os.path.join(chpt_dir, name+'.h5')
        if not os.path.exists(chpt_dir):
            os.makedirs(chpt_dir)
#         Critic Network 1
        self.layers.append(
            nn.Linear(state_dims + action_dims, fc1_dim)
        )
        self.layers.append(
            nn.Linear(fc1_dim, fc2_dim)
        )
        self.layers.append(
            nn.Linear(fc2_dim, 1)
        )
#         Critic Network 2
        self.layers.append(
            nn.Linear(state_dims + action_dims, fc1_dim)
        )
        self.layers.append(
            nn.Linear(fc1_dim, fc2_dim)
        )
        self.layers.append(
            nn.Linear(fc2_dim, 1)
        )
        self.to(device)
    
    def forward(self, state, action):
        input = T.cat([state, action], 1)

        q1 = F.relu(self.layers[0](input))
        q1 = F.relu(self.layers[1](q1))
        q1 = self.layers[2](q1)

        q2 = F.relu(self.layers[3](input))
        q2 = F.relu(self.layers[4](q2))
        q2 = self.layers[5](q2)

        return q1, q2

    def Q1(self, state, action):
        q1 = F.relu(self.layers[0](T.cat([state, action], 1)))
        q1 = F.relu(self.layers[1](q1))
        q1 = self.layers[2](q1)

        return q1

In [12]:
class TD3_DDPG_Agent(Agent):
    def __init__(self, state_dims, action_dims, name = "",
                 alpha = 1e-4, beta = 1e-3, gamma = 0.99,  tau =  0.005, replay_size = pow(10, 6),
                 fc1_dim = 400, fc2_dim = 300, batch_size = 100,
                 theta = 0.15, sigma = 0.2,
                 update_actor_interval = 2, warmup = 1000, warmup_noise = 0.1,
                 env=None):
        #         DDPG
        self.memory = FIFO_Buffer(replay_size, state_dims, action_dims)
        self.gamma = gamma
        self.tau = tau
        self.batch_size = batch_size
        self.max_action = env.action_space.high[0]
        self.min_action = env.action_space.low[0]
#         TD3 specific Variables
        self.n_actions = action_dims
        self.update_actor_interval = update_actor_interval
        self.warmup = warmup
        self.train_step_cnt = 0
        self.time_step = 0
        
        self.noise = OrnsteinUhlenbeckProcess(size = action_dims, theta=theta, mu=0, sigma=sigma)
        self.warmup_noise = warmup_noise
        
#         Checkpoints Folder
        self.dir = name+"_td3_ddpg"
        if not os.path.exists(self.dir):
            os.makedirs(self.dir)        
        
#         Create Networks
        self.actor = Actor_Default(
            state_dims, action_dims, fc1_dim, fc2_dim, self.max_action, name="actor", chpt_dir=self.dir
        )
        self.critic1 = Critic_Default(
            state_dims + action_dims, fc1_dim, fc2_dim, 1, name="critic1", chpt_dir=self.dir
        )
        self.critic2 = Critic_Default(
            state_dims + action_dims, fc1_dim, fc2_dim, 1, name="critic2", chpt_dir=self.dir
        )
        
        self.target_actor = Actor_Default(
            state_dims, action_dims, fc1_dim, fc2_dim, self.max_action, name="target_actor", chpt_dir=self.dir
        )
        self.target_critic1 = Critic_Default(
            state_dims + action_dims, fc1_dim, fc2_dim, 1, name="target_critic1", chpt_dir=self.dir
        )
        self.target_critic2 = Critic_Default(
            state_dims + action_dims, fc1_dim, fc2_dim, 1, name="target_critic2", chpt_dir=self.dir
        )
        
        default_network_initialization(self.actor.layers)
        default_network_initialization(self.critic1.layers)
        default_network_initialization(self.critic2.layers)
        default_network_initialization(self.target_actor.layers)
        default_network_initialization(self.target_critic1.layers)
        default_network_initialization(self.target_critic2.layers)
        
#         Create Optimizer
        self.actor_optimizer = T.optim.Adam(self.actor.parameters(), lr=alpha)
        self.critic1_optimizer = T.optim.Adam(self.critic1.parameters(), lr=beta)
        self.critic2_optimizer = T.optim.Adam(self.critic2.parameters(), lr=beta)
        
#         Init target networks
        self.update_target_networks(tau = 1)
    
    def update_target_networks(self, tau=None):
        if tau is None:
            tau = self.tau

        for param, target_param in zip(self.actor.parameters(), self.target_actor.parameters()):
            target_param.data.copy_(tau * param.data + (1. - tau) * target_param.data)
        
        for param, target_param in zip(self.critic1.parameters(), self.target_critic1.parameters()):
            target_param.data.copy_(tau * param.data + (1. - tau) * target_param.data)
          
        for param, target_param in zip(self.critic2.parameters(), self.target_critic2.parameters()):
            target_param.data.copy_(tau * param.data + (1. - tau) * target_param.data)
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.store(state, action, reward, next_state, done)
        
    def select_action(self, state):
        if self.time_step < self.warmup:
            actions = np.random.normal(scale=self.warmup_noise, size=(self.n_actions,))
        else:
#             Compute action from state
            state = T.tensor(state, dtype=T.float).to(device)
            actions = self.actor.forward(state).cpu().data.numpy().flatten()
#             Add noise
            actions += self.noise.sample()
            
        actions = np.clip(actions, self.min_action, self.max_action)
  
        self.time_step += 1
        return actions
    
    def train(self):      
#         Sample from Replay buffer
        state, action, reward, next_state, not_done = self.memory.get_sameples_tensor(self.batch_size)

        with T.no_grad():
            noise = (T.randn_like(action) * 0.2).clamp(-0.5, 0.5)
            
            next_action = (self.target_actor(next_state) + noise).clamp(self.min_action, self.max_action)
            next_action = T.clamp(next_action, self.min_action, self.max_action)
            
            target_Q1 = self.target_critic1(next_state, next_action)
            target_Q2 = self.target_critic2(next_state, next_action)
            target_Q = T.min(target_Q1, target_Q2)
            
            target = reward + (self.gamma * target_Q * not_done).detach()
            
#         Critic Update
        current_Q1 = self.critic1(state, action)
        current_Q2 = self.critic2(state, action)
        critic_loss = F.mse_loss(target_Q, current_Q1) + F.mse_loss(target_Q, current_Q2)
        
#         Optimization
        self.critic1_optimizer.zero_grad()
        self.critic2_optimizer.zero_grad()
        critic_loss.backward()
        self.critic1_optimizer.step()
        self.critic2_optimizer.step()
    
#         Check if this is the step where we update our policy
        self.train_step_cnt +=1
        if self.train_step_cnt % self.update_actor_interval == 0:
#             Actor Update
            actor_loss = -self.critic1(state, self.actor(state)).mean()

#             Optimization
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()
        
#             Soft update on Target Network
            self.update_target_networks()
        
    def save_models(self):
        self.actor.save()
        self.target_actor.save()
        self.critic1.save()
        self.critic2.save()
        self.target_critic1.save()
        self.target_critic2.save()
        
    def load_models(self):
        self.actor.load()
        self.target_actor.load()
        self.critic1.load()
        self.critic2.load()
        self.target_critic1.load()
        self.target_critic2.load()

In [18]:
# Environment
ENV_NAME = 'InvertedPendulumPyBulletEnv-v0'
env = gym.make(ENV_NAME)
env.seed(0)
# Agent    
agent = TD3_DDPG_Agent(
    name = ENV_NAME,
    state_dims = env.observation_space.shape[0], env=env, action_dims=env.action_space.shape[0],
    fc1_dim = 64, fc2_dim = 64
)
# Run Test
test_agent(env, agent, 100000, False)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
2021-01-28 16:08:40.860034 Total T: 50885 Episode Num: 5043 Episode T: 7 Reward: 7.000
2021-01-28 16:08:41.202207 Total T: 50896 Episode Num: 5044 Episode T: 11 Reward: 11.000
2021-01-28 16:08:41.442058 Total T: 50904 Episode Num: 5045 Episode T: 8 Reward: 8.000
2021-01-28 16:08:41.609485 Total T: 50910 Episode Num: 5046 Episode T: 6 Reward: 6.000
2021-01-28 16:08:41.786295 Total T: 50916 Episode Num: 5047 Episode T: 6 Reward: 6.000
2021-01-28 16:08:42.056544 Total T: 50926 Episode Num: 5048 Episode T: 10 Reward: 10.000
2021-01-28 16:08:42.358222 Total T: 50937 Episode Num: 5049 Episode T: 11 Reward: 11.000
2021-01-28 16:08:42.620515 Total T: 50947 Episode Num: 5050 Episode T: 10 Reward: 10.000
2021-01-28 16:08:42.875754 Total T: 50956 Episode Num: 5051 Episode T: 9 Reward: 9.000
2021-01-28 16:08:43.045059 Total T: 50963 Episode Num: 5052 Episode T: 7 Reward: 7.000
2021-01-28 16:08:43.278901 Total T: 50972 Episode Num: 5053 Episode T: 

In [19]:
from google.colab import files
files.download(agent.dir+'/performance.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>